In [1]:
transaction1 = {
  'amount': '30',
  'sender': 'Alice',
  'receiver': 'Bob'}
transaction2 = { 
  'amount': '200',
  'sender': 'Bob',
  'receiver': 'Alice'}
transaction3 = { 
  'amount': '300',
  'sender': 'Alice',
  'receiver': 'Timothy' }
transaction4 = { 
  'amount': '300',
  'sender': 'Rodrigo',
  'receiver': 'Thomas' }
transaction5 = { 
  'amount': '200',
  'sender': 'Timothy',
  'receiver': 'Thomas' }
transaction6 = { 
  'amount': '400',
  'sender': 'Tiffany',
  'receiver': 'Xavier' }

mempool = [transaction1, transaction2, transaction3, transaction4, transaction5, transaction6]

# add your code below
my_transaction = {
  'amount':'400',
  'sender':'Fiffany',
  'receiver':'peter'
}
mempool.append(my_transaction)
block_transactions=mempool[:3]

In [2]:
#The creation of the block
from datetime import datetime
from hashlib import sha256
class Block:
    #Initializing the block
    def __init__(self, transactions, previous_hash,nonce=0):
        self.transactions = transactions
        self.previous_hash = previous_hash
        self.nonce = nonce
        self.timestamp = datetime.now()
        #Call the geretate_hash to get the hash of block
        self.hash = self.generate_hash()
    def display_block_info(self):
        print("tmestamp: ", self.timestamp)
        print("transactions: ", self.transactions)
        print("Current hash: ", self.generate_hash())
    #We now proceed to generate hash for the block
    def generate_hash(self):
        block_contents = str(self.timestamp)+str(self.transactions)+str(self.nonce)+str(self.previous_hash)
        block_hash = sha256(block_contents.encode())
        return block_hash.hexdigest()


In [3]:
class Blockchain:
    def __init__(self):
        self.chain = []
        self.all_transactions = []
        self.genesis_block()
    
    def genesis_block(self):
        transactions = {}
        genesis_block = Block(transactions, "0")
        self.chain.append(genesis_block)
        return self.chain
    def print_blocks(self):
        for i in range(len(self.chain)):
            current_block = self.chain[i]
            print("Block {} {}".format(i, current_block))
            current_block.display_block_info()
    #Add block to the chain
    def add_block(self, transactions):
        self.previous_block_hash = self.chain[len(self.chain)-1].hash
        new_block = Block(transactions, self.previous_block_hash)
        #We need to calculate the proof of work
        proof = self.proof_of_work(new_block, difficulty=2)
        self.chain.append(new_block)
        return proof, new_block
    def validate_chain(self):
        for i in range(1, len(self.chain)):
            current = self.chain[i]
            previous = self.chain[i-1]
            if (current.hash != current.generate_hash()):
                print("The current hash of the block does not equal the generated hash of the block.")
                return False
            if (current.previous_hash != previous.generate_hash()):
                print("Previous block not equal previous hash value stored in current block")
                return False
        return True
    def proof_of_work(self,block,difficulty=2):
        proof = block.generate_hash()
        while proof[:difficulty] != '0'*difficulty:
            block.nonce += 1
            proof = block.generate_hash()
        block.nonce = 0
        return proof

In [4]:
new_transactions = [{'amount': '30', 'sender':'alice', 'receiver':'bob'},
               	{'amount': '55', 'sender':'bob', 'receiver':'alice'}]

In [5]:
my_blockchain = Blockchain()

In [6]:
my_blockchain.add_block(new_transactions)

('007777041cd737bf3e8cc5f366d8c332e401fb9c8dd15588245c8d7c9600e353',
 <__main__.Block at 0x26cc7808af0>)

In [7]:
my_blockchain.print_blocks()

Block 0 <__main__.Block object at 0x0000026CC78084C0>
tmestamp:  2022-05-13 08:56:16.870069
transactions:  {}
Current hash:  ef71e5f17e6c4cb2a72c1ccf9387d4beff41a05c4a4225d1e5cfc01335d152ee
Block 1 <__main__.Block object at 0x0000026CC7808AF0>
tmestamp:  2022-05-13 08:56:29.335011
transactions:  [{'amount': '30', 'sender': 'alice', 'receiver': 'bob'}, {'amount': '55', 'sender': 'bob', 'receiver': 'alice'}]
Current hash:  9bcef99050508d891904548882103f5b3d097ca4e3439281c4179039edb28bde


In [8]:
my_blockchain.chain[1].transactions = "fake_transactions"

In [9]:
my_blockchain.validate_chain()

The current hash of the block does not equal the generated hash of the block.


False

In [ ]:
class Validator:
    def __init__(self, pub_key):
        